# Exploring building virtual datasets with kerchunk

The idea: build a test intake catalogue of kerchunk virtual datasets from the COSIMA outputs and see how it performs

In [1]:
import dask

import glob

import fsspec

import ujson

import xarray as xr

from distributed import Client

from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr

In [2]:
client = Client(n_workers=4)
print(f"Dask dashboard at: {client.dashboard_link}")

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39811,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:37031,Total threads: 1
Dashboard: http://127.0.0.1:43703/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:35191,


# First try build a virtual dataset from `access-om2-025` `025deg_jra55_iaf_omip2_cycle1` `ocean_month` data (~2.2TB, 61 netcdf files)

## Write single file jsons in parallel

In [3]:
exp_root = "/g/data/ik11/outputs/access-om2-025/025deg_jra55_iaf_omip2_cycle1"

fs = fsspec.filesystem('file')
flist = fs.glob(f"{exp_root}/output*/ocean/ocean_month.nc")

In [4]:
@dask.delayed
def gen_json(file):
    with fs.open(file) as infile:
        h5chunks = SingleHdf5ToZarr(infile, file)
        outf = f"{'.'.join(file.split('/')[5:])}.json"
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [5]:
%%time

# This would take well over an hour without dask
_ = dask.compute(*[gen_json(file) for file in flist])

CPU times: user 1min 57s, sys: 22.7 s, total: 2min 20s
Wall time: 25min 50s


## Combine into multi-file json

In [6]:
%%time

json_list = fs.glob("./access-om2-025.025deg_jra55_iaf_omip2_cycle1.*.ocean.ocean_month.nc.json")

mzz = MultiZarrToZarr(
    json_list[:40],
    concat_dims=['time'],
    identical_dims=[
        "xt_ocean", 
        "yt_ocean", 
        "st_ocean", 
        "xu_ocean", 
        "yu_ocean", 
        "sw_ocean", 
        "grid_xt_ocean", 
        "grid_yt_ocean", 
        "grid_xu_ocean", 
        "grid_yu_ocean", 
        "potrho", 
        "neutral",
        "nv"
     ],
)

d = mzz.translate("access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json")

CPU times: user 1min 7s, sys: 24.3 s, total: 1min 32s
Wall time: 1min 34s


In [7]:
%%time

m = fsspec.get_mapper(
    'reference://', 
    fo="access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json", 
    remote_protocol="file"
)
ds = xr.open_dataset(
    m,
    engine='zarr', 
    backend_kwargs={"consolidated": False},
    chunks={},
    decode_times=False
)

CPU times: user 14 s, sys: 2.22 s, total: 16.2 s
Wall time: 17.6 s


In [8]:
ds

<xarray.Dataset>
Dimensions:                    (time: 480, st_ocean: 50, yt_ocean: 1080,
                                xt_ocean: 1440, yu_ocean: 1080, xu_ocean: 1440,
                                sw_ocean: 50, grid_xt_ocean: 1440,
                                grid_xu_ocean: 1440, grid_yt_ocean: 1080,
                                grid_yu_ocean: 1080, neutral: 80,
                                neutralrho_edges: 81, nv: 2, potrho: 80,
                                potrho_edges: 81, st_edges_ocean: 51,
                                sw_edges_ocean: 51)
Coordinates: (12/18)
  * grid_xt_ocean              (grid_xt_ocean) float64 -279.9 -279.6 ... 79.88
  * grid_xu_ocean              (grid_xu_ocean) float64 -279.8 -279.5 ... 80.0
  * grid_yt_ocean              (grid_yt_ocean) float64 -81.08 -80.97 ... 89.95
  * grid_yu_ocean              (grid_yu_ocean) float64 -81.02 -80.92 ... 90.0
  * neutral                    (neutral) float64 1.028e+03 ... 1.038e+03
  * neutralrho_edges           (neutralrho_edges) float64 1.028e+03 ... 1.038...
    ...                         ...
  * sw_ocean                   (sw_ocean) float64 2.303 4.994 ... 5.363e+03
  * time                       (time) float64 7.148e+05 7.148e+05 ... 7.294e+05
  * xt_ocean                   (xt_ocean) float64 -279.9 -279.6 ... 79.62 79.88
  * xu_ocean                   (xu_ocean) float64 -279.8 -279.5 ... 79.75 80.0
  * yt_ocean                   (yt_ocean) float64 -81.08 -80.97 ... 89.84 89.95
  * yu_ocean                   (yu_ocean) float64 -81.02 -80.92 ... 89.89 90.0
Data variables: (12/102)
    age_global                 (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    average_DT                 (time) float64 dask.array<chunksize=(480,), meta=np.ndarray>
    average_T1                 (time) float64 dask.array<chunksize=(480,), meta=np.ndarray>
    average_T2                 (time) float64 dask.array<chunksize=(480,), meta=np.ndarray>
    bih_fric_u                 (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    bih_fric_v                 (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    ...                         ...
    v                          (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    vert_pv                    (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    vsq                        (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
    wfiform                    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 216, 240), meta=np.ndarray>
    wfimelt                    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 216, 240), meta=np.ndarray>
    wt                         (time, sw_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 25, 108, 120), meta=np.ndarray>
Attributes:
    filename:   ocean_month.nc
    grid_tile:  1
    grid_type:  mosaic
    history:    20201103T0215Z: /g/data/ik11/inputs/access-om2/bin/mppnccombi...
    title:      ACCESS-OM2

In [ ]:
%%time

global_mean = ds["sst"].mean(["xt_ocean", "yt_ocean"])
_ = global_mean.plot()

## Thoughts

- This approach should be relatively straight-forward if we can have some confidence that all input files will be readily "concatenatable" in the sense that they have the same chunking patterns, coordinates on fixed dims, fixed auxiliary coordinates, variables (where approapriate) etc.
- Where the raw data are unchunked (e.g. netcdf3), this approach can significantly impact performance since fsspec will make the full range request even for subsets (https://github.com/fsspec/kerchunk/issues/124)
- Relevant to the previous two points, kerchunk devs are discussing ways to split/combine chunks within a reference file (e.g. so that netcdf files with different native chunking patterns could be easily combined) but this hasn't been implemented and would have limitations (e.g. couldn't work for particular types of compression. See https://github.com/fsspec/kerchunk/issues/124 and https://github.com/fsspec/kerchunk/issues/134